In [1]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [2]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], int(tag))

In [3]:
# Read in the data
train = list(read_dataset("data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [4]:
print(train[2])

([57, 58, 59, 60, 19, 61, 37, 62, 63, 19, 64, 65, 66, 26, 19, 64, 67, 68, 69, 5, 1, 70, 63, 71, 1, 72, 73, 74, 75, 1, 76, 26, 77, 26, 78, 37, 1, 79, 33], 3)


In [5]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [6]:
# Define the model
W_sm = model.add_lookup_parameters((nwords, ntags)) # Word weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [7]:
# A function to calculate scores for one sentence
def calc_scores(words):
  dy.renew_cg()
  score = dy.esum([dy.lookup(W_sm, x) for x in words])
  b_sm_exp = dy.parameter(b_sm)
  return score + b_sm_exp

In [8]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=nan, time=0.39s


ValueError: attempt to get argmax of an empty sequence